<a href="https://colab.research.google.com/github/Jitmandal051004/Rice_type_Classification/blob/main/Rice_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Initialization


In [1]:
!pip install opendatasets --quiet
import opendatasets as od
od.download("https://www.kaggle.com/datasets/mssmartypants/rice-type-classification")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: jitmandal24
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/mssmartypants/rice-type-classification


100%|██████████| 888k/888k [00:00<00:00, 88.1MB/s]

In [2]:
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader
from torchsummary import summary
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


### Pre_processing

In [3]:
data_df = pd.read_csv("/content/rice-type-classification/riceClassification.csv")
data_df.head()

,id,Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,EquivDiameter,Extent,Perimeter,Roundness,AspectRation,Class
0,1,4537,92.229316,64.012769,0.719916,4677,76.004525,0.657536,273.085,0.764510,1.440796,1
1,2,2872,74.691881,51.400454,0.725553,3015,60.471018,0.713009,208.317,0.831658,1.453137,1
2,3,3048,76.293164,52.043491,0.731211,3132,62.296341,0.759153,210.012,0.868434,1.465950,1
3,4,3073,77.033628,51.928487,0.738639,3157,62.551300,0.783529,210.657,0.870203,1.483456,1
4,5,3693,85.124785,56.374021,0.749282,3802,68.571668,0.769375,230.332,0.874743,1.510000,1


In [4]:
data_df.dropna(inplace=True)
data_df.drop(['id'], axis=1, inplace=True)
print(data_df.shape)

(18185, 11)


In [5]:
data_df.head()

,Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,EquivDiameter,Extent,Perimeter,Roundness,AspectRation,Class
0,4537,92.229316,64.012769,0.719916,4677,76.004525,0.657536,273.085,0.764510,1.440796,1
1,2872,74.691881,51.400454,0.725553,3015,60.471018,0.713009,208.317,0.831658,1.453137,1
2,3048,76.293164,52.043491,0.731211,3132,62.296341,0.759153,210.012,0.868434,1.465950,1
3,3073,77.033628,51.928487,0.738639,3157,62.551300,0.783529,210.657,0.870203,1.483456,1
4,3693,85.124785,56.374021,0.749282,3802,68.571668,0.769375,230.332,0.874743,1.510000,1


In [6]:
print(data_df["Class"].unique())
print(data_df["Class"].value_counts())

[1 0]
Class
1    9985
0    8200
Name: count, dtype: int64


### Normalization

In [7]:
orig_data_df = data_df.copy()

for col in data_df.columns:
  data_df[col] = (data_df[col] / data_df[col].abs().max())

data_df.head()

,Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,EquivDiameter,Extent,Perimeter,Roundness,AspectRation,Class
0,0.444368,0.503404,0.775435,0.744658,0.424873,0.666610,0.741661,0.537029,0.844997,0.368316,1.0
1,0.281293,0.407681,0.622653,0.750489,0.273892,0.530370,0.804230,0.409661,0.919215,0.371471,1.0
2,0.298531,0.416421,0.630442,0.756341,0.284520,0.546380,0.856278,0.412994,0.959862,0.374747,1.0
3,0.300979,0.420463,0.629049,0.764024,0.286791,0.548616,0.883772,0.414262,0.961818,0.379222,1.0
4,0.361704,0.464626,0.682901,0.775033,0.345385,0.601418,0.867808,0.452954,0.966836,0.386007,1.0


In [8]:
X = np.array(data_df.iloc[:, :-1]) #all values except last col
Y = np.array(data_df.iloc[:, -1]) #last col

print(X.shape, Y.shape)

(18185, 10) (18185,)


### Splitting of the dataset

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5)

print(X_train.shape, X_test.shape, X_val.shape)

(12729, 10) (2728, 10) (2728, 10)


### Dataset object


In [10]:
class dataset(Dataset):
  def __init__(self,X,Y):
    self.X = torch.tensor(X, dtype=torch.float32).to(device) #vector
    self.Y = torch.tensor(Y, dtype=torch.float32).to(device)

  def __len__(self):
    return len(self.X)

  def __getitem__(self, index):
    return self.X[index], self.Y[index]

In [11]:
training_data = dataset(X_train, y_train)
testing_data = dataset(X_test, y_test)
validation_data = dataset(X_val, y_val)

### Data_loader


In [34]:
train_dataloader = DataLoader(training_data, batch_size=8, shuffle=True)
validation_dataloader = DataLoader(validation_data, batch_size=8, shuffle=True)
testing_dataloader = DataLoader(testing_data, batch_size=8, shuffle=True)

In [35]:
for x, y in train_dataloader:
  print(x)
  print("========")
  print(y)
  break

tensor([[0.8759, 0.8784, 0.8733, 0.9248, 0.8470, 0.9359, 0.6868, 0.7624, 0.8264,
         0.5706],
        [0.9236, 0.8869, 0.9104, 0.9171, 0.8807, 0.9610, 0.8156, 0.7743, 0.8448,
         0.5527],
        [0.4781, 0.6522, 0.6383, 0.9284, 0.4538, 0.6914, 0.7080, 0.5516, 0.8616,
         0.5797],
        [0.4616, 0.7187, 0.5668, 0.9668, 0.4424, 0.6794, 0.5444, 0.5874, 0.7338,
         0.7193],
        [0.9096, 0.8604, 0.9234, 0.9054, 0.8696, 0.9537, 0.6659, 0.7585, 0.8670,
         0.5286],
        [0.8649, 0.8610, 0.8791, 0.9184, 0.8269, 0.9300, 0.6445, 0.7462, 0.8519,
         0.5557],
        [0.7395, 0.7935, 0.8078, 0.9191, 0.7011, 0.8599, 0.8388, 0.7003, 0.8269,
         0.5573],
        [0.5912, 0.8395, 0.6194, 0.9755, 0.5665, 0.7689, 0.5383, 0.6868, 0.6874,
         0.7689]], device='cuda:0')
tensor([0., 0., 1., 1., 0., 0., 0., 1.], device='cuda:0')


### model

In [16]:
HIDDEN_NEURONS = 10

In [18]:
class MyModel(nn.Module):
  def __init__(self):
    super(MyModel, self).__init__() #here we will use the constructor of the nn module

    self.input_layer = nn.Linear(X.shape[1], HIDDEN_NEURONS)
    self.linear = nn.Linear(HIDDEN_NEURONS, 1)
    self.sigmoid = nn.Sigmoid()

  def forward(self, x): #define the flow of the data in the model
    x = self.input_layer(x)
    x = self.linear(x)
    x = self.sigmoid(x)
    return x

model = MyModel().to(device)

### Create Summary


In [20]:
summary(model, (X.shape[1],)) #-1 shows batch size

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                   [-1, 10]             110
            Linear-2                    [-1, 1]              11
           Sigmoid-3                    [-1, 1]               0
Total params: 121
Trainable params: 121
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00
----------------------------------------------------------------


### Loss and Optimizer



In [21]:
criterion = nn.BCELoss() #Binary Cross Entropy Loss
optimizer = Adam(model.parameters(), lr=1e-3)

In [37]:
total_loss_train_plot = []
total_loss_val_plot = []
total_acc_train_plot = []
total_acc_val_plot = []

epochs = 10
for epoch in range(epochs):
  total_loss_train = 0
  total_acc_train = 0
  total_loss_val = 0
  total_acc_val = 0

  for data in train_Dataloader:
    inputs, labels = data # batch size * batch size

    prediction = model(inputs).squeeze(1) #convert torch.Size([8, 1]) to torch.Size([8]) if used 1

    #loss
    batch_loss = criterion(prediction, labels) # loss after every each epoch
    total_loss_train += batch_loss.item() # after all epoch

    #accuracy
    acc = ((prediction).round() == labels).sum().item() #.item() convert tensor to main item
    total_acc_train += acc

    batch_loss.backward() #from other side
    optimizer.step() #initiate and steps optimmizer to change the weight and restart again, prevent non-convergences
    optimizer.zero_grad()

  with torch.no_grad(): #prepare the model for prediction without changing the model perimeters
    for data in validation_dataloader:
      inputs, labels = data

      prediction = model(inputs).squeeze(1)

      #loss
      batch_loss = criterion(prediction, labels)
      total_loss_val += batch_loss.item() # after all epoch

      #accuracy
      acc = ((prediction).round() == labels).sum().item() #.item() convert tensor to main item
      total_acc_val += acc
    total_loss_train_plot.append(round(total_loss_train / 1000, 4)) #round fn
    total__loss_val_plot.append(round(total_val_train / 1000, 4)) #used to rounding

    total_acc_train_plot.append(round(total_acc_train / training_data.__len__() * 100, 4))
    total_acc_val_plot.append(round(total_acc_val / 1000, 4))

TypeError: 'DataLoader' object is not subscriptable